In [1]:
import os

def find_xes_files(starting_folder):
    result = []

    for root, dirs, files in os.walk(starting_folder):
        for file in files:
            if file.endswith('.xes'):
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, starting_folder)
                parts = relative_path.split(os.sep)
                if len(parts) > 1:
                    dataset_name = parts[0]
                else:
                    dataset_name = os.path.splitext(file)[0]
                result.append({
                    'dataset_name': dataset_name,
                    'dataset_path': file_path
                })

    return result

all_files = find_xes_files('/local/s3377954/remote_ssh_files/raw_eventlogs')
all_files

In [4]:
from event_log_stats.dataset_generator import DatasetGenerator
import concurrent.futures


def process_file(file):
    print(file)
    generator = DatasetGenerator('plot_data_day')
    generator.generate(file['dataset_name'], file['dataset_path'], 'day')
    
with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(process_file, file) for file in all_files]
    for future in concurrent.futures.as_completed(futures):
        future.result()